# Generating visualization images and graphics for the bubble/shock wave interaction simulation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imageio
from mpl_toolkits.axes_grid1 import make_axes_locatable
import progressbar

plt.ioff()

# parametres
mx = 2001      # x mesh
my = 201       # y mash
Db = 0.004     # bubble diameter
pas_t = 0.005  # time step (in e-6 s)
nmax = 18000   # maximum iteration number
kres = 2000    # frequence of outputs
M = 1.73       # Mach

Uflow = {'1.18':93.6482,
         '1.24':125.1538,
         '1.73':333.5403,
         '2.00':433.8496,
         '3.00':771.1353,
         '4.00':1084.2369}

U = Uflow[f'{M:.2f}'] # m/s

n = int(nmax / kres)  # number of files

mx = mx - 4  # mx - 2 results in x and we'll exclude the two last ones (outliers)
my = my - 2  # my - 2 results in y
t = kres * pas_t  # time between images
t0 = - 0.001/(346.956*M) * 1e6    # initial time at the interaction shock/bubble
T = Db / U * 1e6    # constant of time
print('Dimensionless time step: ', t/T)

source_path = f'../Output_SCB_M{M:.2f}'
dest_vis2D = 'images/M{M:.2f}/vis2D'
dest_graphs = 'images/M{M:.2f}/graphs'

Dimensionless time step:  0.83385075


#### Parameters for simulations with the same dimensionless time step:

1. M = 1.18:
    - nmax = 77000
    - kres = 90
    - t/T = 0.010535422499999999
2. M = 1.73:
    - nmax = 24000
    - kres = 25
    - t/T = 0.010423134374999999
3. M = 2.00:
    - nmax = 17252
    - kres = 19
    - t/T = 0.010303928
4. M = 3.00:
    - nmax = 9790
    - kres = 11
    - t/T = 0.010603110375
5. M = 4.00:
    - nmax = 69966
    - kres = 78
    - t/T = 0.010571309775000001

## Visualisation 2D

In [2]:
parameters = ["alpha", "Pstat", "rho", "gradrho", "u", "v"]

vmin = {}
vmax = {}

for parameter in parameters:
    vmin[parameter] = np.inf
    vmax[parameter] = -np.inf
    
units = {'alpha':'',
         'Pstat':'bar',
         'rho':'kg/m³',
         'gradrho':'kg/m³',
         'u':'m/s',
         'v':'m/s'}

### Finding the global min and max for each parameter to scale the graphs

In [3]:
bar = progressbar.ProgressBar(max_value=n)

for i in range(1,n+1):
    filename = 'SCB_output'
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'

    data = pd.read_csv(f'{source_path}/{filename}.data',
                       sep="\s+", header=None,
                       names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                       low_memory=False)
    
    for column in data.columns:
        if data[column].dtypes != 'float64':
            data[column] = pd.to_numeric(data[column], errors='coerce')

    data = data.replace(np.nan, 0)
    
    data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
    data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)
    
    for parameter in parameters:
        local_min = data[f'{parameter}'].min()
        local_max = data[f'{parameter}'].max()
        if local_min < vmin[f'{parameter}']:
            vmin[f'{parameter}'] = local_min
        if local_max > vmax[f'{parameter}']:
            vmax[f'{parameter}'] = local_max
    
    bar.update(i)

100% (9 of 9) |##########################| Elapsed Time: 0:00:04 ETA:  00:00:00

#### $\rho$ min and max among all Machs for comparison

In [4]:
#vmin = {'rho':0.160}
#vmax = {'rho':6.000}  # 1.967, 4.100, 5.000, 6.688, 8.215

### Generating visualization images

#### Single parameters

In [5]:
parameters = ["alpha", "Pstat", "rho"]

for i in range(1,n+1):
    filename = 'SCB_output'
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'

    data = pd.read_csv(f'{source_path}/{filename}.data',
                       sep="\s+", header=None,
                       names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                       low_memory=False)
    
    for column in data.columns:
        if data[column].dtypes != 'float64':
            data[column] = pd.to_numeric(data[column], errors='coerce')

    data = data.replace(np.nan, 0)
    
    data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
    data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)
    
    # Admensionalizing lenghts
    
    data['x'] = data['x'] / Db
    data['y'] = data['y'] / Db
    
    # Individual images for each parameter
    
    for parameter in parameters:
        variable = data[f'{parameter}'].to_numpy(copy=True)
        variable = variable.reshape((my,mx))
        
        plt.figure(figsize=(16,4))
        plt.rcParams.update({'font.size': 16})
        ax = plt.subplot()
        plt.title(f'M = {M:.2f}, t = {t0+i*t:.1f} $\mu$s, T = {(t0+i*t)/T:.2f}')
        im = ax.imshow(variable, origin='lower', cmap='plasma',
                       extent=[data['x'].min(),
                               data['x'].max(),
                               data['y'].min(),
                               data['y'].max()],
                       vmin=vmin[f'{parameter}'],
                       vmax=vmax[f'{parameter}'])

        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.1)

        ticks=[(5-i)/5*vmin[f'{parameter}']+i/5*vmax[f'{parameter}'] for i in range(6)]
        plt.colorbar(im, cax=cax, ticks=ticks).set_label(units[f'{parameter}'])

        plt.savefig(f'{dest_vis2D}/vis2D_{parameter}_t{i*t:05.1f}.png', dpi=180)
        plt.close()

#### Two parameters together

##### $\alpha$ and $P_{stat}$

In [6]:
parameter1 = 'alpha'
parameter2 = 'Pstat'
bar = progressbar.ProgressBar(max_value=n)

for i in range(1,n+1):
    filename = 'SCB_output'
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'

    data = pd.read_csv(f'{source_path}/{filename}.data',
                       sep="\s+", header=None,
                       names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                       low_memory=False)
    
    for column in data.columns:
        if data[column].dtypes != 'float64':
            data[column] = pd.to_numeric(data[column], errors='coerce')

    data = data.replace(np.nan, 0)
    
    data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
    data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)
        
    data['x'] = data['x'] / Db
    data['y'] = data['y'] / Db
    
    # Selecting parameters 1 and 2
    
    par1 = data[f'{parameter1}'].to_numpy(copy=True)
    par1 = par1.reshape((my,mx))
    
    par2 = data[f'{parameter2}'].to_numpy(copy=True)
    par2 = par2.reshape((my,mx))
    
    # Plot

    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(16,5.5)) #(16,5)
    fig.suptitle(f'M = {M:.2f}, t = {t0+i*t:.1f} $\mu$s, T = {(t0+i*t)/T:.2f}')
    plt.subplots_adjust(hspace=-0.432)  # -0.0 bigger domaine
    plt.rcParams.update({'font.size': 16})
    
    ## Subplot 1 - parameter 1
    ax1.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax1.set_ylabel(f'{parameter1}', rotation='vertical', fontsize=12)
    im1 = ax1.imshow(par1, origin='lower', cmap='plasma',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin[f'{parameter1}'],
                     vmax=vmax[f'{parameter1}'])
    
    ### Colorbar
    divider1 = make_axes_locatable(ax1)
    cax1 = divider1.append_axes("top", size="15%", pad="8%")
    
    ticks=[(7-i)/7*vmin[f'{parameter1}']+i/7*vmax[f'{parameter1}'] for i in range(8)]
    fig.colorbar(im1, cax=cax1, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter1}'], labelpad=-55)
    cax1.xaxis.set_ticks_position("top")
    
    ## Subplot 2 - parameter 2
    ax2.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax2.set_ylabel(f'{parameter2}', rotation='vertical', fontsize=12)
    im2 = ax2.imshow(par2, origin='lower', cmap='viridis',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin[f'{parameter2}'],
                     vmax=vmax[f'{parameter2}'])
    ax2.invert_yaxis()
    
    ### Colorbar
    divider2 = make_axes_locatable(ax2)
    cax2 = divider2.append_axes("bottom", size="15%", pad="8%")

    ticks=[(7-i)/7*vmin[f'{parameter2}']+i/7*vmax[f'{parameter2}'] for i in range(8)]
    fig.colorbar(im2, cax=cax2, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter2}'])
    
    ## Saving
    plt.savefig(f'{dest_vis2D}/vis2D_{parameter1}&{parameter2}_t{i*t:05.1f}.png', dpi=180)
    plt.close()
    
    bar.update(i)

100% (9 of 9) |##########################| Elapsed Time: 0:00:07 ETA:  00:00:00

#### $\rho$ and $\nabla \rho$

In [7]:
parameter1 = 'rho'
parameter2 = 'gradrho'
bar = progressbar.ProgressBar(max_value=n)

for i in range(1,n+1):
    filename = 'SCB_output'
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'

    data = pd.read_csv(f'{source_path}/{filename}.data',
                       sep="\s+", header=None,
                       names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                       low_memory=False)
    
    for column in data.columns:
        if data[column].dtypes != 'float64':
            data[column] = pd.to_numeric(data[column], errors='coerce')

    data = data.replace(np.nan, 0)
    
    data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
    data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)
        
    data['x'] = data['x'] / Db
    data['y'] = data['y'] / Db
    
    # Selecting parameters 1 and 2
    
    par1 = data[f'{parameter1}'].to_numpy(copy=True)
    par1 = par1.reshape((my,mx))
    
    par2 = data[f'{parameter2}'].to_numpy(copy=True)
    par2 = par2.reshape((my,mx))
    
    # Plot

    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(16,5.5))
    fig.suptitle(f'M = {M:.2f}, t = {t0+i*t:.1f} $\mu$s, T = {(t0+i*t)/T:.2f}')
    plt.subplots_adjust(hspace=-0.432)
    plt.rcParams.update({'font.size': 16})
    
    ## Subplot 1 - parameter 1
    ax1.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax1.set_ylabel(f'{parameter1}', rotation='vertical', fontsize=12)
    im1 = ax1.imshow(par1, origin='lower', cmap='plasma',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin[f'{parameter1}'],
                     vmax=vmax[f'{parameter1}'])
    
    ### Colorbar
    divider1 = make_axes_locatable(ax1)
    cax1 = divider1.append_axes("top", size="15%", pad="8%")
    
    ticks=[(7-i)/7*vmin[f'{parameter1}']+i/7*vmax[f'{parameter1}'] for i in range(8)]
    fig.colorbar(im1, cax=cax1, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter1}'], labelpad=-55)
    cax1.xaxis.set_ticks_position("top")
    
    ## Subplot 2 - parameter 2
    ax2.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax2.set_ylabel(f'{parameter2}', rotation='vertical', fontsize=12)
    im2 = ax2.imshow(par2, origin='lower', cmap='viridis',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin[f'{parameter2}'],
                     vmax=vmax[f'{parameter2}'])
    ax2.invert_yaxis()
    
    ### Colorbar
    divider2 = make_axes_locatable(ax2)
    cax2 = divider2.append_axes("bottom", size="15%", pad="8%")

    ticks=[(7-i)/7*vmin[f'{parameter2}']+i/7*vmax[f'{parameter2}'] for i in range(8)]
    fig.colorbar(im2, cax=cax2, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter2}'])
    
    ## Saving
    plt.savefig(f'{dest_vis2D}/vis2D_{parameter1}&{parameter2}_t{i*t:05.1f}.png', dpi=180)
    plt.close()
    
    bar.update(i)

100% (9 of 9) |##########################| Elapsed Time: 0:00:07 ETA:  00:00:00

#### $u$ and $v$

In [8]:
parameter1 = 'u'
parameter2 = 'v'
bar = progressbar.ProgressBar(max_value=n)

for i in range(1,n+1):
    filename = 'SCB_output'
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'

    data = pd.read_csv(f'{source_path}/{filename}.data',
                       sep="\s+", header=None,
                       names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                       low_memory=False)
    
    for column in data.columns:
        if data[column].dtypes != 'float64':
            data[column] = pd.to_numeric(data[column], errors='coerce')

    data = data.replace(np.nan, 0)
    
    data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
    data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)
        
    data['x'] = data['x'] / Db
    data['y'] = data['y'] / Db
    
    # Selecting parameters 1 and 2
    
    par1 = data[f'{parameter1}'].to_numpy(copy=True)
    par1 = par1.reshape((my,mx))
    
    par2 = data[f'{parameter2}'].to_numpy(copy=True)
    par2 = par2.reshape((my,mx))
    
    # Plot

    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(16,5.5))
    fig.suptitle(f'M = {M:.2f}, t = {t0+i*t:.1f} $\mu$s, T = {(t0+i*t)/T:.2f}')
    plt.subplots_adjust(hspace=-0.432)
    plt.rcParams.update({'font.size': 16})
    
    ## Subplot 1 - parameter 1
    ax1.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax1.set_ylabel(f'{parameter1}', rotation='vertical', fontsize=12)
    im1 = ax1.imshow(par1, origin='lower', cmap='plasma',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin[f'{parameter1}'],
                     vmax=vmax[f'{parameter1}'])
    
    ### Colorbar
    divider1 = make_axes_locatable(ax1)
    cax1 = divider1.append_axes("top", size="15%", pad="8%")
    
    ticks=[(7-i)/7*vmin[f'{parameter1}']+i/7*vmax[f'{parameter1}'] for i in range(8)]
    fig.colorbar(im1, cax=cax1, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter1}'], labelpad=-55)
    cax1.xaxis.set_ticks_position("top")
    
    ## Subplot 2 - parameter 2
    ax2.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax2.set_ylabel(f'{parameter2}', rotation='vertical', fontsize=12)
    im2 = ax2.imshow(par2, origin='lower', cmap='viridis',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin[f'{parameter2}'],
                     vmax=vmax[f'{parameter2}'])
    ax2.invert_yaxis()
    
    ### Colorbar
    divider2 = make_axes_locatable(ax2)
    cax2 = divider2.append_axes("bottom", size="15%", pad="8%")

    ticks=[(7-i)/7*vmin[f'{parameter2}']+i/7*vmax[f'{parameter2}'] for i in range(8)]
    fig.colorbar(im2, cax=cax2, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter2}'])
    
    ## Saving
    plt.savefig(f'{dest_vis2D}/vis2D_{parameter1}&{parameter2}_t{i*t:05.1f}.png', dpi=180)
    plt.close()
    
    bar.update(i)

100% (9 of 9) |##########################| Elapsed Time: 0:00:07 ETA:  00:00:00

#### $\rho$ and $P_{stat}$

In [9]:
parameter1 = 'rho'
parameter2 = 'Pstat'
bar = progressbar.ProgressBar(max_value=n)

for i in range(1,n+1):
    filename = 'SCB_output'
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'

    data = pd.read_csv(f'{source_path}/{filename}.data',
                       sep="\s+", header=None,
                       names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                       low_memory=False)
    
    for column in data.columns:
        if data[column].dtypes != 'float64':
            data[column] = pd.to_numeric(data[column], errors='coerce')

    data = data.replace(np.nan, 0)
    
    data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
    data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)
        
    data['x'] = data['x'] / Db
    data['y'] = data['y'] / Db
    
    # Selecting parameters 1 and 2
    
    par1 = data[f'{parameter1}'].to_numpy(copy=True)
    par1 = par1.reshape((my,mx))
    
    par2 = data[f'{parameter2}'].to_numpy(copy=True)
    par2 = par2.reshape((my,mx))
    
    # Plot

    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(16,5.5))
    fig.suptitle(f'M = {M:.2f}, t = {t0+i*t:.1f} $\mu$s, T = {(t0+i*t)/T:.2f}')
    plt.subplots_adjust(hspace=-0.432)
    plt.rcParams.update({'font.size': 16})
    
    ## Subplot 1 - parameter 1
    ax1.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax1.set_ylabel(f'{parameter1}', rotation='vertical', fontsize=12)
    im1 = ax1.imshow(par1, origin='lower', cmap='plasma',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin[f'{parameter1}'],
                     vmax=vmax[f'{parameter1}'])
    
    ### Colorbar
    divider1 = make_axes_locatable(ax1)
    cax1 = divider1.append_axes("top", size="15%", pad="8%")
    
    ticks=[(7-i)/7*vmin[f'{parameter1}']+i/7*vmax[f'{parameter1}'] for i in range(8)]
    fig.colorbar(im1, cax=cax1, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter1}'], labelpad=-55)
    cax1.xaxis.set_ticks_position("top")
    
    ## Subplot 2 - parameter 2
    ax2.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    ax2.set_ylabel(f'{parameter2}', rotation='vertical', fontsize=12)
    im2 = ax2.imshow(par2, origin='lower', cmap='viridis',
                     extent=[data['x'].min(),
                             data['x'].max(),
                             data['y'].min(),
                             data['y'].max()],
                     vmin=vmin[f'{parameter2}'],
                     vmax=vmax[f'{parameter2}'])
    ax2.invert_yaxis()
    
    ### Colorbar
    divider2 = make_axes_locatable(ax2)
    cax2 = divider2.append_axes("bottom", size="15%", pad="8%")

    ticks=[(7-i)/7*vmin[f'{parameter2}']+i/7*vmax[f'{parameter2}'] for i in range(8)]
    fig.colorbar(im2, cax=cax2, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter2}'])
    
    ## Saving
    plt.savefig(f'{dest_vis2D}/vis2D_{parameter1}&{parameter2}_t{i*t:05.1f}.png', dpi=180)
    plt.close()
    
    bar.update(i)

100% (9 of 9) |##########################| Elapsed Time: 0:00:07 ETA:  00:00:00

#### Complete bubble

In [10]:
parameter = 'rho'
bar = progressbar.ProgressBar(max_value=n)

for i in range(1,n+1):
    filename = 'SCB_output'
    if i != n:
        filename += f'{i:05}'
    else:
        filename += '_final'

    data = pd.read_csv(f'{source_path}/{filename}.data',
                       sep="\s+", header=None,
                       names=["x", "y", "alpha", "Pstat", "rho", "gradrho", "u", "v"],
                       low_memory=False)
    
    for column in data.columns:
        if data[column].dtypes != 'float64':
            data[column] = pd.to_numeric(data[column], errors='coerce')

    data = data.replace(np.nan, 0)
    
    data.drop(data[data['x']==data['x'].iloc[mx+1]].index, inplace=True)
    data.drop(data[data['x']==data['x'].iloc[mx]].index, inplace=True)
    
    
    # Selecting parameters 1 and 2
    
    par1 = data[f'{parameter}'].to_numpy(copy=True)
    par1 = par1.reshape((my,mx))
    
    par2 = np.flip(par1, axis=0)
    
    variable = np.append(par2, par1, axis=0)
    
    # Plot

    plt.figure(figsize=(16,5))
    plt.rcParams.update({'font.size': 16})
    ax = plt.subplot()
    ax.tick_params(bottom=False, labelbottom='off', top=False, labeltop='off', left=False, labelleft='off')
    plt.title(f'M = {M:.2f}, t = {t0+i*t:.1f} $\mu$s, T = {(t0+i*t)/T:.2f}')
    im = ax.imshow(variable, origin='lower', cmap='plasma',
                   extent=[data['x'].min(),
                           data['x'].max(),
                           -data['y'].max(),
                           data['y'].max()],
                   vmin=vmin[f'{parameter}'],
                   vmax=vmax[f'{parameter}'])
    
    ## Colorbar
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("bottom", size="15%", pad="8%")

    ticks=[(7-i)/7*vmin[f'{parameter}']+i/7*vmax[f'{parameter}'] for i in range(8)]
    plt.colorbar(im, cax=cax, orientation='horizontal', ticks=ticks).set_label(units[f'{parameter}'])

    ## Saving
    
    plt.savefig(f'{dest_vis2D}/vis2D_complet_{parameter}_t{i*t:05.1f}.png', dpi=180)
    plt.close()
    
    bar.update(i)

100% (9 of 9) |##########################| Elapsed Time: 0:00:06 ETA:  00:00:00

### Generating gifs

In [11]:
parameters = ["complet_rho"] #"alpha", "Pstat", "rho", "alpha&Pstat", "rho&Pstat"]

filenames = {}

for parameter in parameters:
    filenames[parameter] = []

for i in range(1,n+1):
    for parameter in parameters:
        filenames[f'{parameter}'].append(f'vis2D_{parameter}_t{i*t:05.1f}.png')
        
for parameter in parameters:
    bar = progressbar.ProgressBar(max_value=len(filenames[f'{parameter}']))
    with imageio.get_writer(f'{dest_vis2D}/gifs/M{M:.2f}_{parameter}.gif', mode='I', duration = 1/16) as writer:
        for filename in filenames[f'{parameter}']:
            image = imageio.imread(f'{dest_vis2D}/{filename}')
            writer.append_data(image)
            bar.update(filenames[f'{parameter}'].index(filename)+1)

100% (9 of 9) |##########################| Elapsed Time: 0:00:02 ETA:  00:00:00

___
## Graphics

In [12]:
parameters = ['alpha']
             #'bubble_len','cson','density','fractiony',
             #'pmax','pressure','pwall_max','residu_final','temperature',
             #'umax','velocity','vol_vap','wall_pressure'

for parameter in parameters:
    if parameter in ['bubble_len','pmax','pwall_max','residu_final','umax','vol_vap']:
        pass
    else:
        for i in range(1,n+1):
            filename = f'SCB_{parameter}'
            if i != n:
                filename += f'{i:05}'
            else:
                filename += '_final'

            data = pd.read_csv(f'{source_path}/{filename}.data', sep="\s+", header=None, names=['x',parameter])
            data.drop(data.tail(2).index, inplace=True)
            data.plot(x='x', y=parameter, ylabel=parameter, legend=False)
            plt.savefig(f'{dest_graphs}/graph_M{M:.2f}_{parameter}_t{i*t:05.1f}.png')
            plt.close()

### Generating gifs

In [13]:
parameters = ['alpha']
             #'cson','density','fractiony',
             #'pressure','temperature',
             #'velocity','wall_pressure'

filenames = {}

for parameter in parameters:
    filenames[parameter] = []

for i in range(1,n+1):
    for parameter in parameters:
        filenames[f'{parameter}'].append(f'graph_M{M:.2f}_{parameter}_t{i*t:05.1f}.png')

with imageio.get_writer(f'{dest_graphs}/gifs/graph_M{M:.2f}_{parameter}.gif', mode='I', duration = 0.2) as writer:
    for filename in filenames[f'{parameter}']:
        image = imageio.imread(f'{dest_graphs}/{filename}')
        writer.append_data(image)